In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text # Not used directly but needed to import TF ops.

from modules.evaluation import calculate_embeddings, Index
from tqdm import tqdm

ModuleNotFoundError: No module named 'modules'

In [2]:
df = pd.read_parquet('data/qq_sim.parquet')
questions = df['question'].unique()
model = hub.KerasLayer("models/USEm_large3")

In [87]:
questions = df2['question'].unique()

In [88]:
quest_vectors = {}
embs = calculate_embeddings(model, questions, dims=512, model_type='use')

Calculating embeddings: 100%|██████████| 328/328 [26:52<00:00,  4.92s/it]


In [100]:
for i, question in tqdm(enumerate(questions)):
    quest_vectors.update({question: embs[i]})

21018it [00:00, 491645.00it/s]


In [21]:
df2 = pd.read_parquet('data/qa_clean.parquet')
answers = df2['answer'].unique()

In [22]:
ans_vectors = {}
embs = calculate_embeddings(model, answers, dims=512, model_type='use')
for i, answer in tqdm(enumerate(answers)):
    ans_vectors.update({answer: embs[i]})

Calculating embeddings: 100%|██████████| 771/771 [33:55<00:00,  2.64s/it]
49394it [00:00, 1078578.99it/s]


In [24]:
import pickle
with open('data/not_load/questions_dict.pkl', 'wb') as f:
    pickle.dump(quest_vectors, f)
with open('data/not_load/answers_dict.pkl', 'wb') as f:
    pickle.dump(ans_vectors, f)

In [29]:
search_index = Index(model, questions, 512, model_type='use')

Calculating embeddings: 100%|██████████| 80/80 [06:13<00:00,  4.66s/it]


In [50]:
sample = df2[df2['channel'] == 'datasets'].sample()
sample['question'].to_numpy()[0]

'а кроме wiki, какие существуют хорошие датасеты для multilabel text classification?'

In [51]:
candidates, _ = search_index.search(quest_vectors[sample['question'].to_numpy()[0]].reshape(1, -1))

In [52]:
candidates

['Привет! А кто-нибудь знает датасеты для классификации текстов на русском языке? Хотелось бы что-то типа 20newsgroups, но только для великого и могучего. П.С. знаю, что можно намайнить, но лень)',
 'всем привет!\nкто-нибудь знает какие-нибудь статьи/ссылки по сравнению (по любой метрике) различных моделей для классификации текста? идеально было бы для IMDB датасета',
 'Привет. Есть ли где-нибудь датасет размеченных текстов на русском языке по темам?',
 'Господа а какие есть тулзы упрощающие разметку текстовых датасетов?',
 'Привет, пацаны. Мне нужен очень большой датасет текстов на русском языке, типа всей русскоязычной википедии. Может быть знаете, кто-то уже выкачивал такое и где можно это найти?',
 'Всем привет. А может у кого-то есть классификатор товаров по категориям на русском или может какой-нибудь датасет подходящий знаете?',
 'кто-нибудь знает какие сейчас существуют интересные SOTA (но необязательно) методы для content labeling? То есть присваивать человечески читаемые кате

In [53]:
query_answers = df2[df2['question'] == sample['question'].to_numpy()[0]]['answer'].to_numpy(); query_answers

array(['arxiv, еще на гитхабе лейблы появились, может быть можно где-то дамп найти',
       'я в <#C0KL2AXD3|datasets> выкладывал 1.5m статей с medium, там есть теги.',
       'около 2000 вроде.',
       'stackexchange по разным темам вопросы с тегами'], dtype=object)

In [54]:
candidates_answers = []
for cand in candidates:
    candidates_answers.append(df2[df2['question'] == cand]['answer'].to_numpy())

candidates_answers

[array(['тут <https://tatianashavrina.github.io/taiga_site/downloads> есть новости',
        'Есть вот такой для чатботов <https://toloka.yandex.ru/datasets/>',
        '<https://github.com/buriy/russian-nlp-datasets/releases/tag/r4>'],
       dtype=object),
 array(['<https://paperswithcode.com/sota/sentiment-analysis-on-imdb>\n\n<https://paperswithcode.com/sota>',
        '<https://ahmedbesbes.com/overview-and-benchmark-of-traditional-and-deep-learning-models-in-text-classification.html>',
        '<http://nlpprogress.com/english/sentiment_analysis.html>'],
       dtype=object),
 array(['polo: Как вариант, можешь распарсить новостные источники. Вот, например, я парсил Ленту <https://github.com/yutkin/lenta.ru-news-dataset>.'],
       dtype=object),
 array(['<http://brat.nlplab.org/>'], dtype=object),
 array(['Ссылку не доставлю, под рукой нет. Но гуглить по типу wiki dump',
        'Википедия маленькая, но если нужна -- погугли по этому слаку. Дамп бери cirrussearch, иначе шаблоны буд

In [55]:
from scipy.spatial.distance import cosine

table = []
for ans in query_answers:
    a = []
    for cands in candidates_answers:
        c = []
        for cand in cands:
            c.append(cosine(ans_vectors[ans], ans_vectors[cand]))
        a.append(c)
    table.append(a)

In [56]:
from pprint import pprint

pprint(table)

[[[0.8172813802957535, 0.9083745554089546, 0.8315234333276749],
  [0.9691755156964064, 1.007233098614961, 1.0146066658198833],
  [0.9150481224060059],
  [0.9349732547998428],
  [0.8696997463703156, 0.7376111447811127, 0.9362394958734512],
  [0.832336887717247],
  [0.5044894218444824],
  [0.9202183857560158],
  [0.8712908327579498, 0.8729972988367081],
  [0.9000867381691933]],
 [[0.7655701637268066, 0.7902668118476868, 0.7274530529975891],
  [0.7958397567272186, 0.8685403913259506, 0.8821618929505348],
  [0.6144543588161469],
  [0.8915234208106995],
  [0.6944743990898132, 0.6880724728107452, 0.8641466796398163],
  [0.7651455849409103],
  [0.7577382773160934],
  [0.7824581116437912],
  [0.7887020111083984, 0.6699787676334381],
  [0.8169232308864594]],
 [[0.9631971195340157, 0.9263541921973228, 0.9901438821107149],
  [0.970547741279006, 0.9943454670719802, 0.9420468583703041],
  [1.0014581942232326],
  [0.8481171280145645],
  [0.8794928044080734, 0.9135652184486389, 0.9912729542702436],
 

От ответов

In [57]:
search_idx = Index(model, answers, 512, model_type='use')

Calculating embeddings: 100%|██████████| 771/771 [46:03<00:00,  3.58s/it]


In [90]:
def _try_ans():
    sample = df2[df2['channel'] == 'datasets'].sample()
    print(f"Random answer: {sample['answer'].to_numpy()[0]}")
    candidates, _ = search_idx.search(ans_vectors[sample['answer'].to_numpy()[0]].reshape(1, -1))

    candidates_questions = []
    for cand in candidates:
        candidates_questions.append(df2[df2['answer'] == cand]['question'].to_numpy())

    table = []
    q = sample['question'].to_numpy()[0]
    for cands in candidates_questions:
        for cand in cands:
            try:
                table.append((cand, cosine(quest_vectors[q], quest_vectors[cand])))
            except KeyError:
                continue

    print(f'Question for that answer: {q}\nCandidates:\n')
    pprint(table)

In [74]:
sample = df2[df2['channel'] == 'datasets'].sample()
sample['answer'].to_numpy()[0]

'microsoft malware classification challenge'

In [75]:
candidates, _ = search_idx.search(ans_vectors[sample['answer'].to_numpy()[0]].reshape(1, -1))

In [77]:
candidates_questions = []
for cand in candidates:
    candidates_questions.append(df2[df2['answer'] == cand]['question'].to_numpy())

#candidates_questions

In [78]:
table = []
q = sample['question'].to_numpy()[0]
for cands in candidates_questions:
    for cand in cands:
        try:
            table.append((cand, cosine(quest_vectors[q], quest_vectors[cand])))
        except KeyError:
            continue

print(q)
pprint(table)

Добрый вечер! Подскажите, пожалуйста: где можно достать датасет по malware? Данные статического (тип файла, результат выполнения strings, импорты dll и т.д.) или динамического (какая команда в какой момент времени выполнялась, создаваемые процессы и т.д.) анализа какой-нибудь выборки вирусов или другого вредоносного по. Цель - анализ данных при помощи визуализации. Заранее спасибо :slightly_smiling_face:
[('А можете посоветовать датасет для определения парафраз? датасет от quora '
  'уже видел, интересует что-то другое',
  0.6053649485111237)]


In [105]:
_try_ans()

Random answer: Там больше 800 тысяч словоформ, и, конечно же, это делалось скриптом :slightly_smiling_face: А в качестве алгоритма g2p использовался <https://github.com/nsu-ai/russian_g2p>
Вручную сделанные транскрипции тоже есть, но их совсем немного, несколько сотен, и они сделаны для аббревиатур типа "мгу" и подобных
Question for that answer: Всем привет! Нет ли у кого нибудь датасета grapheme2phoneme на русском языке?
Candidates:

[('*TLDR: Какие NLP модели подходят мобильным платформам?*\n'
  '\n'
  'Взращиваю сад моделей TF.js и вижу, что представлено не так много сетей, '
  'работающих с языком: есть Universal Sentence Encoder '
  '(<https://github.com/tensorflow/tfjs-models/tree/master/universal-sentence-encoder>) '
  'с вокабуляром 8к и результирующими эмбеддингами в 512, есть классификатор '
  'токсика, есть распознователь голосовых команд. Была идея портировать HNAtt '
  '(<https://github.com/minqi/hnatt>), но она базируется на Glove, да и '
  'функционал пересекается с токс

In [106]:
channels = ['analytiсs', 'audio_and_speech', 'bayesian', 'big_data', 'bioinformatics', 'blockchain', 'cloud', 'datasets',
            'deep_learning', 'devops', 'gan', 'gis', 'lang_cpp', 'lang_go', 'lang_javascript', 'lang_julia', 'lang_python'
            'lang_r', 'lang_scala', 'math', 'nlp', 'recommender_systems', 'reinforcement_learning', 'risk_modelling', 
            'satellite_imaging', 'trading', 'visualization']

df = df2[df2['channel'].isin(channels)]

In [107]:
df_new = pd.DataFrame(columns=['question', 'similiar_questions'])

In [111]:
df_new['question'] = df['question']

In [123]:
str(similiar_questions).strip('[]')

''

In [128]:
list_for_clusterization = []
append_flag = False
for i, row in tqdm(df.iterrows(), total=len(df)):
    question = row['question']
    similiar_questions = []
    answer = row['answer']
    candidates, _ = search_idx.search(ans_vectors[answer].reshape(1, -1))

    candidates_questions = []
    for cand in candidates:
        candidates_questions.append(df[df['answer'] == cand]['question'].to_numpy())

    for cand_array in candidates_questions:
        for cand in cand_array:
            try:
                distance = cosine(quest_vectors[question], quest_vectors[cand])
                if all((distance <= 0.5, distance > 0)):
                    similiar_questions.append(cand)
                    list_for_clusterization.append(cand)
                    append_flag = True
            except KeyError:
                continue

    if append_flag:
        list_for_clusterization.append(question)
        append_flag = False
    
    similiar_questions = str(similiar_questions).strip('[]')
    if similiar_questions == '':
        similiar_questions = np.nan
    df_new.loc[i, 'similiar_questions'] = similiar_questions

100%|██████████| 10333/10333 [10:01<00:00, 17.17it/s]


In [129]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10333 entries, 0 to 48559
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   question            10333 non-null  object
 1   similiar_questions  680 non-null    object
dtypes: object(2)
memory usage: 562.2+ KB


In [133]:
df_new[df_new['similiar_questions'].notna()].drop_duplicates('question')

,question,similiar_questions
120,"Подскажите, что обычно используют для разметки...","'Ребят, какой посоветуете удобный инструмент д..."
124,"Вопрос может показаться туповатым, но я раньше...",'Здравствуйте. Есть ли в Python Matlab-like ви...
285,Кто сталкивался c OCR? Есть ли что-то готовое ...,"'чат, а есть что-то более актуальное про OCR с..."
296,"Я понимаю, что уже всех заколебал, но пока чат...","'Ребят, задача бинарной сегментации :resnet34:..."
306,"Какая ваша любимая библиотека для pytorch, обл...","""Вопрос к любителям :pytorch1-0: Как вы обучае..."
...,...,...
46873,"Подскажите, пожалуйста, чем рисуют архитектуры...","'Народ, а кто какие тулы использует для рисова..."
46878,доброго дня. У меня есть некий датасет с двумя...,'Здравствуйте. Есть ли в Python Matlab-like ви...
46885,Если бы у вас внезапно проступило желание забо...,"'Друзья, привет! Есть ли какая то питоновская ..."
46905,"Хей. Есть функция: r = sin(a)*cos(a*3/2), где ...","'Кто-то знает, как сделать чтобы графики не на..."


In [135]:
df2

,channel,question,answer,reactions
0,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","Шатать таблички в го это, как летать на дачу н...","[{'name': 'heavy_plus_sign', 'count': 1}]"
1,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",Зачем эта простыня текста. Ее традиционно никт...,"[{'name': 'tnx', 'count': 1}, {'name': 'heavy_..."
2,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",> в проект Data Science\nвыглядит как bullshit...,"[{'name': '+1', 'count': 1}]"
3,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",<@U64GCA997> сильно лучше :notbad:,"[{'name': 'tnx', 'count': 1}]"
4,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",Что такое бустануть? Что за нужно уметь в них?...,"[{'name': 'heavy_plus_sign', 'count': 1}, {'na..."
...,...,...,...,...
49389,proj_kaggle_2sigma,"Как то можно писать в лог в процессе commit, ч...",можно `logging` юзать. тогда можно одно и тоже...,"[{'name': 'tnx', 'count': 1}]"
49390,proj_kaggle_2sigma,"Как то можно писать в лог в процессе commit, ч...",например так:\n```# Set up a logger to dump me...,"[{'name': '+1', 'count': 1}]"
49391,proj_kaggle_2sigma,"Ребят, а подскажите, я некоторое время назад с...","насколько я понимаю, после того как закончится...","[{'name': 'heavy_plus_sign', 'count': 2}]"
49392,proj_kaggle_2sigma,"Ребят, а подскажите, я некоторое время назад с...",1. Решения нет - это вообще бесспорно. 4 - наи...,"[{'name': '+1', 'count': 1}, {'name': 'heavy_p..."


In [142]:
cluster_df = pd.DataFrame(columns=['question', 'channel'])
cluster_df['question'] = list_for_clusterization
cluster_df.drop_duplicates('question', inplace=True)
for i, row in tqdm(cluster_df.iterrows(), total=len(cluster_df)):
    row['channel'] = df2[df2['question'] == row['question']]['channel'].to_numpy()[0]

cluster_df

100%|██████████| 665/665 [00:03<00:00, 173.22it/s]


,question,channel
0,"Ребят, какой посоветуете удобный инструмент дл...",deep_learning
1,"Подскажите, что обычно используют для разметки...",gis
2,Здравствуйте. Есть ли в Python Matlab-like виз...,visualization
3,"Вопрос может показаться туповатым, но я раньше...",gis
6,"чат, а есть что-то более актуальное про OCR се...",deep_learning
...,...,...
1483,"Друзья, привет! Подскажите пожалуйста каким об...",visualization
1489,"Друзья, привет! сейчас будет нубский вопрос, a...",visualization
1513,"Друзья, привет! Есть ли какая то питоновская б...",visualization
1514,Если бы у вас внезапно проступило желание забо...,visualization


In [143]:
cluster_df.to_parquet('data/not_load/sim_clusters.parquet', compression='brotli', index=False)

In [144]:
df_new.to_parquet('data/not_load/qq_sim_from_use.parquet', compression='brotli', index=False)

In [3]:
df = pd.read_parquet('../data/not_load/qq_sim_from_use.parquet')

In [14]:
df.dropna(inplace=True); df

,question,similiar_questions
62,"Подскажите, что обычно используют для разметки...","'Ребят, какой посоветуете удобный инструмент д..."
66,"Вопрос может показаться туповатым, но я раньше...",'Здравствуйте. Есть ли в Python Matlab-like ви...
67,"Вопрос может показаться туповатым, но я раньше...",'Здравствуйте. Есть ли в Python Matlab-like ви...
108,Кто сталкивался c OCR? Есть ли что-то готовое ...,"'чат, а есть что-то более актуальное про OCR с..."
119,"Я понимаю, что уже всех заколебал, но пока чат...","'Ребят, задача бинарной сегментации :resnet34:..."
...,...,...
10163,"Подскажите, пожалуйста, чем рисуют архитектуры...","'Привет, часто встречаю диаграммы архитектур н..."
10166,доброго дня. У меня есть некий датасет с двумя...,'Здравствуйте. Есть ли в Python Matlab-like ви...
10173,Если бы у вас внезапно проступило желание забо...,"'Друзья, привет! Есть ли какая то питоновская ..."
10193,"Хей. Есть функция: r = sin(a)*cos(a*3/2), где ...","'Кто-то знает, как сделать чтобы графики не на..."


In [17]:
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

In [19]:
df.loc[13, 'question']

'Всем привет. Хочу перекатиться с :keras: на :pytorch1-0:. Есть какие нибудь гайды (на пример как создавать `dataloader`и подобное), чтоб быстро вкатиться. Или сразу использовать :catalyst:?'

In [20]:
df.loc[13, 'similiar_questions']

"':catalyst: :keras: А какие аналоги Catalyst есть для Keras?\\n\\nНужна штука которая позволяет красиво и удобно настроить кучу моделей и гиперпараметров, отправить это дело учиться и собирать метрики, чтобы потом любоваться на графики и выбирать самое сочное'"

In [24]:
len(df['question'].unique())

546